In [1]:
import torch
import XRDXRFutils as xutils
from glob import glob
from src import nets, dataset
import yaml
from torch.utils.data import Dataset,DataLoader,WeightedRandomSampler
import numpy as np
from matplotlib import pyplot as plt

In [2]:
# load the model
model_path = glob('./run/synth_sampled_CNN2L1Loss16_ch_run_0/*.pth')[-1]
with open('./run/synth_CNN2L1Loss16_ch_run_0/config.yaml','r') as file:
    config = yaml.load(file,Loader=yaml.FullLoader)
model = nets.CNN2(channels=config['channels'], n_outputs=10)

In [3]:
#load Data for the evaluation
data2evaluate = xutils.DataXRF().load_h5('F:/andrea/AIdataXRF/base/2156HVIIAnalysisBisShiftCorrected_$_weighted_x1024_with_labels.h5')
data2evaluate = data2evaluate.select_labels(config['labels'])

Loading: F:/andrea/AIdataXRF/base/2156HVIIAnalysisBisShiftCorrected_$_weighted_x1024_with_labels.h5
Adding empty labels: ['Co-K', 'Ti-K']


In [4]:
print(data2evaluate.labels.shape)
data = torch.from_numpy(data2evaluate.data).reshape(-1,1,data2evaluate.data.shape[-1]).float()
labels = torch.from_numpy(data2evaluate.labels).reshape(-1,data2evaluate.labels.shape[-1]).float()
ds = dataset.SpectraDataset(data=data, labels=labels)
checkpoint = torch.load(model_path)
model.load_state_dict(checkpoint['model_state_dict'], strict=False)
model.eval()

(838, 638, 10)


CNN2(
  (conv_0): Conv1d(1, 16, kernel_size=(5,), stride=(1,), padding=(2,), bias=False)
  (conv_1): Conv1d(16, 32, kernel_size=(5,), stride=(1,), padding=(2,), bias=False)
  (conv_2): Conv1d(32, 64, kernel_size=(5,), stride=(1,), padding=(2,), bias=False)
  (conv_3): Conv1d(64, 128, kernel_size=(5,), stride=(1,), padding=(2,), bias=False)
  (c0): Sequential(
    (0): Conv1d(1, 16, kernel_size=(5,), stride=(1,), padding=(2,), bias=False)
    (1): LeakyReLU(negative_slope=0.1)
    (2): Conv1d(16, 16, kernel_size=(1,), stride=(2,), bias=False)
  )
  (c1): Sequential(
    (0): Conv1d(16, 32, kernel_size=(5,), stride=(1,), padding=(2,), bias=False)
    (1): LeakyReLU(negative_slope=0.1)
    (2): Conv1d(32, 32, kernel_size=(1,), stride=(2,), bias=False)
  )
  (c2): Sequential(
    (0): Conv1d(32, 64, kernel_size=(5,), stride=(1,), padding=(2,), bias=False)
    (1): LeakyReLU(negative_slope=0.1)
    (2): Conv1d(64, 64, kernel_size=(1,), stride=(2,), bias=False)
  )
  (c3): Sequential(
    (0

In [5]:
evaluate = DataLoader(ds,
                batch_size = config['batch_size'],
                shuffle = False,
                drop_last = False)
evaluate.shape = data2evaluate.labels.shape

In [6]:
print(data2evaluate.data.shape)

(838, 638, 1024)


In [7]:
image = []
device = torch.device(config['device'])
model.to(device)
for i,batch in enumerate(evaluate):

    x,y = batch

    x = x.to(device)
    y = y.to(device)

    outputs = model(x)
    image += [outputs.cpu().detach().numpy()]

image = np.vstack(image)
print('\n',image.shape)
image = image.reshape(evaluate.shape)


 (534644, 10)


In [12]:
%matplotlib qt
labels = data2evaluate.labels
print(data2evaluate.metadata['labels'])
for i, element in enumerate(data2evaluate.metadata['labels']):
    plt.figure()
    ax_model = plt.subplot(221)
    ax_model.set_title('model '+ element)
    ax_model.imshow(image[:,:,i], cmap='copper')

    ax_maxrf = plt.subplot(222)
    ax_maxrf.set_title('label '+ element)
    ax_maxrf.imshow(labels[:,:,i], cmap='copper')
    
    ax_hist = plt.subplot(212)
    ax_hist.set_title('histogram of counts')
    m = np.maximum(np.max(image[:,:,i]), np.max(labels[:,:,i]))
    ry,rx = np.histogram(image[:,:,i].reshape(-1), bins=int(m+1),range=(0, m+1))
    iy,ix = np.histogram(labels[:,:,i].reshape(-1), bins=int(m+1),range=(0, m+1))
    ax_hist.plot(rx[:-1], ry,label=element +' - labels')
    ax_hist.plot(ix[:-1], iy,label=element +' - model', alpha=0.8)
    ax_hist.legend()
    plt.show()

['Pb-L' 'Au-L' 'Ca-K' 'Co-K' 'Cu-K' 'Fe-K' 'Mn-K' 'Hg-L' 'Ti-K' 'Zn-K']
